## Mathematics for AI - Coursework Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0  and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible. You can implement one of the two (or both, for no extra point) game modes:

• The time spent on a cell is the number on this cell

• The time spent on a cell is the absolute of the difference between the previous cell the agent was on and the current cell it is on.

In order to solve this problem, you will provide 3 algorithms:

• A first baseline of your choosing. E.g. it can be any search algorithm, or an algorithm using heuristics. It doesn’t have to perform fast or well, but should be better than random movements.

• Dijkstra'salgorithm

• Ant colony optimization algorithm. You should describe the algorithms and compare them.

Are they always solving the problem? How long do they take depending on the size of the maze?

### Notes
- grid of size **heightxwidth**
    - each cell = random integer from 0-9
- **START** upper-left
- **GOAL** get to lower-right as fast as possible
- number on the cell = time spent on that cell

## Make Grid

In [63]:
# Import (limited) libraries
from random import seed
from random import randint
import numpy as np
import sys 

In [64]:
g = []
width = 10
height = 15
seed(5)

for row in range(height):
    g.append([])
    for column in range(width):
        g[row].append(randint(0,9))

In [65]:
# Convert to numpy.array
grid = np.array(g)
print(grid)

[[9 4 5 8 0 7 3 0 2 1]
 [5 7 3 6 8 1 9 3 0 3]
 [6 4 2 6 2 1 2 9 9 7]
 [2 2 0 0 3 3 2 2 4 5]
 [3 8 3 2 3 6 4 0 5 6]
 [2 2 4 1 5 4 9 9 0 9]
 [5 1 4 5 4 7 5 2 7 7]
 [2 0 4 0 5 6 0 8 6 5]
 [6 9 0 7 0 2 9 3 1 3]
 [7 5 8 5 8 4 7 1 9 5]
 [4 0 6 1 3 5 8 9 5 2]
 [5 4 8 1 4 5 4 2 1 2]
 [4 7 2 0 1 9 8 6 0 3]
 [9 5 4 7 6 2 0 0 7 5]
 [3 2 9 2 6 1 2 6 5 2]]


## Baseline Algorithm

In [82]:
class Baseline():
    
    def __init__(self, graph):
        self.graph = graph
    
    # Algorithm goes down to last row, then right to last column -> finish
    def algorithm(self):
        

        src = [0,0]
        current = src
        timeSpent = 0 # Total time spent by algorithm
        X = current[1]
        y = current[0]
        
        # Go down through rows
        while(X != (height-1)):
            X += 1
            print(X,y)
            timeSpent += self.graph[X,y]; # Value of field added to total
  
        # Go right through columns
        while(y != (width-1)):
            y += 1
            print(X,y)
            timeSpent += self.graph[X,y]; # Value of field added to total
            
        return timeSpent

In [84]:
base = Baseline(grid)
result = base.algorithm()
print(result)

1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
98


In [62]:
'''
visited = []
src = [0,0]
finish = [length-1,length-1]
current = src
timeSpent = 0
        
while current != finish or timeSpent > 3000:
    visited.append(current)
    print(visited)
    values = []
    X = current[1]
    y = current[0]
    timeSpent += grid[X,y]
    locations = [[y, X + 1],[y, X - 1], [y + 1, X],[y - 1, X]]
    for i in locations:
        if(i[0] < 0 or i[1] < 0 or i[0] > length - 1 or i[1] > length - 1):
            locations.remove(i)
    
    for k in locations:
        value = grid[k[0], k[1]]
        values.append(value)
      
    final = dict(zip(values, locations))
    moveTo = 0
    for l in sorted (final.keys()):  
        moveTo = l
        if(l < moveTo):
            moveTo = l
            
    current = final[moveTo]
'''

'\nvisited = []\nsrc = [0,0]\nfinish = [length-1,length-1]\ncurrent = src\ntimeSpent = 0\n        \nwhile current != finish or timeSpent > 3000:\n    visited.append(current)\n    print(visited)\n    values = []\n    X = current[1]\n    y = current[0]\n    timeSpent += grid[X,y]\n    locations = [[y, X + 1],[y, X - 1], [y + 1, X],[y - 1, X]]\n    for i in locations:\n        if(i[0] < 0 or i[1] < 0 or i[0] > length - 1 or i[1] > length - 1):\n            locations.remove(i)\n    \n    for k in locations:\n        value = grid[k[0], k[1]]\n        values.append(value)\n      \n    final = dict(zip(values, locations))\n    moveTo = 0\n    for l in sorted (final.keys()):  \n        moveTo = l\n        if(l < moveTo):\n            moveTo = l\n            \n    current = final[moveTo]\n'

**Description**:

- Assuming the agent can move one cell in 4 directions (LEFT, RIGHT, UP, DOWN)
- Check adjacent cells for the quickest cell, while ignoring already visited cells.
- Move to that cell
- repeat until the end has been reached

This algorithm does not neccesarily get the shortest path to the end, however its pathing is not random as it tries to move to the lowest valued cell adjacent to it (avoiding previously visited cells). 

A simple way that this algorithm could be improved:
When recording the value of the cell, it should be divided by a scalar factor (the closer it is to the goal the smaller the calculation will be); this will reduce the value of the cell that is closer to the end goal and favour it over moving in a direction away from the goal.

## Dijkstra's Algorithm

In [10]:
import sys 
   
class Graph(): 
   
    def __init__(self, vertices): 
        self.V = vertices 
        self.graph = [[0 for column in range(vertices)]  
                    for row in range(vertices)] 
   
    def printSolution(self, dist): 
        print ("Vertex tDistance from Source") 
        for node in range(self.V): 
            print (node, "t", dist[node]) 
   
    # A utility function to find the vertex with  
    # minimum distance value, from the set of vertices  
    # not yet included in shortest path tree 
    def minDistance(self, dist, sptSet): 
   
        # Initilaize minimum distance for next node 
        min = sys.maxsize 
   
        # Search not nearest vertex not in the  
        # shortest path tree 
        for v in range(self.V): 
            if dist[v] < min and sptSet[v] == False: 
                min = dist[v] 
                min_index = v 
   
        return min_index 
   
    # Funtion that implements Dijkstra's single source  
    # shortest path algorithm for a graph represented  
    # using adjacency matrix representation 
    def dijkstra(self, src): 
   
        dist = [sys.maxsize] * self.V 
        dist[src] = 0
        sptSet = [False] * self.V 
   
        for cout in range(self.V): 
   
            # Pick the minimum distance vertex from  
            # the set of vertices not yet processed.  
            # u is always equal to src in first iteration 
            u = self.minDistance(dist, sptSet) 
   
            # Put the minimum distance vertex in the  
            # shotest path tree 
            sptSet[u] = True
   
            # Update dist value of the adjacent vertices  
            # of the picked vertex only if the current  
            # distance is greater than new distance and 
            # the vertex in not in the shotest path tree 
            for v in range(self.V): 
                if self.graph[u][v] > 0 and sptSet[v] == False and dist[v] > dist[u] + self.graph[u][v]: 
                    dist[v] = dist[u] + self.graph[u][v] 
   
        self.printSolution(dist) 
    
g = Graph(length) 
g.graph = grid
   
g.dijkstra(0); 

Vertex tDistance from Source
0 t 0
1 t 5
2 t 3
3 t 4
4 t 2
5 t 2
6 t 8
7 t 5
8 t 4
9 t 5


## Ant Colony Algorithm

- pheromones:     when ants travel they will leave some pheromones behind, the strenght of their pheromones will be calculated.
- vaporisation:   the pheromones will evaporate at a constant rate over time
- multiple ants do the this.
    - the pheromones increase the probability of the ants following that path.
- then the final ant takes the one with the highest pheromones.

